<a href="https://colab.research.google.com/github/SyedaJannatul/NLP-Project--Sentiment-Analysis/blob/main/Sentiment_Analysis_imdb_movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow as tf

  Created wheel for tf: filename=tf-1.0.0-cp37-none-any.whl size=1285 sha256=cddc90e842eac28848bdb096e9a4104da4f3ec08acc6cbf43571d638369ec5ed
  Stored in directory: /root/.cache/pip/wheels/7c/51/42/b034446bd6c8252825a1a0e7799b6ff633de376967db5ed5a6
Successfully built tf


In [2]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [3]:
!pip install -q tensorflow-datasets

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [7]:
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete862OC8/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete862OC8/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete862OC8/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [9]:
import numpy as np

In [10]:
train_data,test_data=imdb['train'],imdb['test']

In [11]:
training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

In [12]:
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())


In [13]:
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [14]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [15]:
print(training_sentences[0:2])

['b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."', "b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot develo

In [16]:
print(training_labels[0:5])

[0, 0, 0, 1, 1]


In [17]:
print(training_labels_final[0:5])


[0 0 0 1 1]


In [18]:
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='pre'
oov_tok="<OOV>"

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
type(training_sentences)

list

In [20]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
train_sequences=tokenizer.texts_to_sequences(training_sentences)
train_padded=pad_sequences(train_sequences,maxlen=max_length,truncating=trunc_type)

In [21]:
test_sequences=tokenizer.texts_to_sequences(testing_sentences)
test_padded=pad_sequences(test_sequences,maxlen=max_length,truncating=trunc_type)

In [22]:
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                           tf.keras.layers.GlobalAveragePooling1D(),
                           tf.keras.layers.Dense(6,activation="relu"),
                           tf.keras.layers.Dense(1,activation="sigmoid")])

In [29]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['acc'])

In [30]:
num_epochs=15
model.fit(train_padded,training_labels_final,epochs=num_epochs,validation_data=(test_padded,testing_labels_final))

Epoch 1/15
782/782 [==============================] - 4s 4ms/step - loss: 0.0308 - acc: 0.9955 - val_loss: 0.8844 - val_acc: 0.8138
Epoch 2/15
782/782 [==============================] - 3s 4ms/step - loss: 0.0290 - acc: 0.9961 - val_loss: 0.8917 - val_acc: 0.8146
Epoch 3/15
782/782 [==============================] - 3s 4ms/step - loss: 0.0282 - acc: 0.9954 - val_loss: 0.8986 - val_acc: 0.8152
Epoch 4/15
782/782 [==============================] - 3s 4ms/step - loss: 0.0276 - acc: 0.9961 - val_loss: 0.9048 - val_acc: 0.8157
Epoch 5/15
782/782 [==============================] - 3s 4ms/step - loss: 0.0257 - acc: 0.9960 - val_loss: 0.9141 - val_acc: 0.8161
Epoch 6/15
782/782 [==============================] - 3s 4ms/step - loss: 0.0289 - acc: 0.9957 - val_loss: 0.9192 - val_acc: 0.8161
Epoch 7/15
782/782 [==============================] - 3s 4ms/step - loss: 0.0240 - acc: 0.9965 - val_loss: 0.9281 - val_acc: 0.8167
Epoch 8/15
782/782 [==============================] - 3s 4ms/step - loss: 0.

In [32]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

In [33]:
def decode_review(text):
  return " ".join([reverse_word_index.get(i,'?') for i in text])

In [34]:
print(decode_review(train_padded[0]))

? ? b this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining <OOV> like christopher <OOV> good name i could barely sit through it


In [35]:
print(training_sentences[0])

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
